In [3]:
import os 
os.chdir("C://Users//dell//Documents//GitHub//Kidney-Tumor-Classification")
%pwd

'C:\\Users\\dell\\Documents\\GitHub\\Kidney-Tumor-Classification'

In [15]:
import dagshub
dagshub.init(repo_owner='saadan1234', repo_name='Kidney-Tumor-Classification', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\dell\miniconda3\envs\kidney\Lib\site-packages\rich\live.py:229: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=5946ae6b-c64c-4661-ac0a-d900577f468d&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=1e1ec96e70004271d1abc398f88ff14aa90a68f64a2bdc929c8d2bab9f38d442




Accessing as saadan1234

Initialized MLflow to track repo "saadan1234/Kidney-Tumor-Classification"

Repository saadan1234/Kidney-Tumor-Classification initialized!

In [16]:
import tensorflow as tf 

model = tf.keras.models.load_model("artifacts/training/model.h5")

In [17]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: int
    params_batch_size: int

In [18]:
from kidneyClassifier import *
from kidneyClassifier.constants import *
from kidneyClassifier.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)-> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion",
            mlflow_uri="https://dagshub.com/saadan1234/Kidney-Tumor-Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [21]:
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [22]:
from kidneyClassifier.utils.common import save_json


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else: 
                mlflow.keras.log_model(self.model, "model")

In [23]:
try: 
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

Found 2207 images belonging to 2 classes.
138/138 ━━━━━━━━━━━━━━━━━━━━ 665s 5s/step - accuracy: 0.9144 - loss: 0.5875


2024/08/29 22:30:34 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
Successfully registered model 'VGG16Model'.
2024/08/29 22:31:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
2024/08/29 22:31:36 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/29 22:31:36 INFO mlflow.tracking._tracking_service.client: 🏃 View run stylish-mule-191 at: https://dagshub.com/saadan1234/Kidney-Tumor-Classification.mlflow/#/experiments/0/runs/24e6f6970ab04136a4bb0167adf2567a.
2024/08/29 22:31:36 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/saadan1234/Kidney-Tumor-Classification.mlflow/#/experiments/0.


In [ ]:
# MLFLOW_TRACKING_URI = https://dagshub.com/saadan1234/Kidney-Tumor-Classification.mlflow
# MLFLOW_TRACKING_USERNAME = saadan1234
# MLFLOW_TRACKING_PASSWORD = 

In [ ]:
# import dagshub
# dagshub.init(repo_owner='saadan1234', repo_name='Kidney-Tumor-Classification', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)